# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [71]:
csv_path = "../output_data/cities.csv"
vacation_df = pd.read_csv(csv_path)
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,37.51,87,75,11.50,CL,1627170947
1,1,hobart,-42.8794,147.3294,44.46,68,40,4.00,AU,1627171035
2,2,saldanha,-33.0117,17.9442,53.73,56,57,11.54,ZA,1627171206
3,3,villa maria,-32.4075,-63.2402,58.60,69,0,5.99,AR,1627171208
4,4,mahebourg,-20.4081,57.7000,67.24,78,24,11.30,MU,1627171209


In [72]:
vacation_spots_df = vacation_df.drop(['Unnamed: 0'], axis=1)
vacation_spots_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.1500,-70.9167,37.51,87,75,11.50,CL,1627170947
1,hobart,-42.8794,147.3294,44.46,68,40,4.00,AU,1627171035
2,saldanha,-33.0117,17.9442,53.73,56,57,11.54,ZA,1627171206
3,villa maria,-32.4075,-63.2402,58.60,69,0,5.99,AR,1627171208
4,mahebourg,-20.4081,57.7000,67.24,78,24,11.30,MU,1627171209


In [73]:
vacation_spots_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,5.630000e+02
mean,20.375226,19.527703,69.297371,71.561279,55.868561,8.092913,1.627171e+09
std,32.848040,92.396810,14.056841,19.383865,38.643186,5.784580,9.197606e+01
min,-54.800000,-175.200000,32.560000,8.000000,0.000000,0.180000,1.627171e+09
25%,-5.166800,-61.266500,59.800000,63.000000,15.500000,3.980000,1.627171e+09
50%,24.344800,22.200000,71.550000,76.000000,67.000000,6.910000,1.627171e+09
75%,47.199650,104.125950,79.835000,85.000000,94.000000,11.365000,1.627171e+09
max,78.218600,179.316700,108.140000,100.000000,100.000000,38.990000,1.627171e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [74]:
gmaps.configure(api_key = g_key)

In [75]:
lat_lng = vacation_spots_df[["Lat", "Lng"]].astype(float)

humidity = vacation_spots_df["Humidity"].astype(float)



In [78]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

humidity_layer = gmaps.heatmap_layer(lat_lng, weights = humidity, 
                                     dissipating = False, max_intensity = 100,
                                     point_radius = 3)
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [89]:
hotel_df = vacation_spots_df.loc[(vacation_spots_df["Wind Speed"] <= 9) & (vacation_spots_df["Cloudiness"]<= 10) & 
                  (vacation_spots_df["Max Temp"] >= 75) & (vacation_spots_df["Max Temp"] <= 85) & (vacation_spots_df["Humidity"] <= 70)].dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,jiexiu,37.0244,111.9125,76.62,61,1,0.92,CN,1627171067
143,urumqi,43.8010,87.6005,77.95,29,0,6.71,CN,1627171276
253,shache,38.4167,77.2406,79.92,30,6,4.65,CN,1627171321
260,tateyama,34.9833,139.8667,84.22,69,0,8.95,JP,1627171325
261,merrill,45.1805,-89.6835,82.81,54,1,5.75,US,1627171326
288,porto walter,-8.2686,-72.7439,75.43,55,8,0.65,BR,1627171336
336,sena madureira,-9.0656,-68.6569,79.83,48,6,0.60,BR,1627171354
376,puerto quijarro,-17.7833,-57.7667,78.03,36,0,2.10,BO,1627171369
416,atasu,48.6814,71.6433,76.55,27,9,7.14,KZ,1627171383
420,mehriz,31.5917,54.4318,81.70,16,0,3.71,IR,1627171385


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,jiexiu,37.0244,111.9125,76.62,61,1,0.92,CN,1627171067,
143,urumqi,43.8010,87.6005,77.95,29,0,6.71,CN,1627171276,
253,shache,38.4167,77.2406,79.92,30,6,4.65,CN,1627171321,
260,tateyama,34.9833,139.8667,84.22,69,0,8.95,JP,1627171325,
261,merrill,45.1805,-89.6835,82.81,54,1,5.75,US,1627171326,
288,porto walter,-8.2686,-72.7439,75.43,55,8,0.65,BR,1627171336,
336,sena madureira,-9.0656,-68.6569,79.83,48,6,0.60,BR,1627171354,
376,puerto quijarro,-17.7833,-57.7667,78.03,36,0,2.10,BO,1627171369,
416,atasu,48.6814,71.6433,76.55,27,9,7.14,KZ,1627171383,
420,mehriz,31.5917,54.4318,81.70,16,0,3.71,IR,1627171385,


In [103]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params = params)
    hotels = hotels.json()
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (KeyError, IndexError):
    print("Missing result.... Skipping.")
hotel_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,jiexiu,37.0244,111.9125,76.62,61,1,0.92,CN,1627171067,
143,urumqi,43.8010,87.6005,77.95,29,0,6.71,CN,1627171276,
253,shache,38.4167,77.2406,79.92,30,6,4.65,CN,1627171321,
260,tateyama,34.9833,139.8667,84.22,69,0,8.95,JP,1627171325,
261,merrill,45.1805,-89.6835,82.81,54,1,5.75,US,1627171326,
288,porto walter,-8.2686,-72.7439,75.43,55,8,0.65,BR,1627171336,
336,sena madureira,-9.0656,-68.6569,79.83,48,6,0.60,BR,1627171354,
376,puerto quijarro,-17.7833,-57.7667,78.03,36,0,2.10,BO,1627171369,
416,atasu,48.6814,71.6433,76.55,27,9,7.14,KZ,1627171383,
420,mehriz,31.5917,54.4318,81.70,16,0,3.71,IR,1627171385,


In [106]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [111]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(marker_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))